In [119]:
import boto3
import pandas as pd
from io import StringIO

In [120]:
# Import dataset
s3_client = boto3.client('s3')

# Load data
# Applicant details
response_1 = s3_client.get_object(Bucket='data-504-final-project-v2', Key='Cleaned/cleaned_applicants_details.csv')
content_1 = response_1['Body'].read().decode('utf-8')
applicants = pd.read_csv(StringIO(content_1))

# Sparta day
response_2 = s3_client.get_object(Bucket='data-504-final-project-v2', Key='Cleaned/combined_sparta_day_test_score.csv')
content_2 = response_2['Body'].read().decode('utf-8')
sparta_day = pd.read_csv(StringIO(content_2))

# Decision scores
response_3 = s3_client.get_object(Bucket='data-504-final-project-v2', Key='Cleaned/cleaned_talent_decision_scores.csv')
content_3 = response_3['Body'].read().decode('utf-8')
decision_scores = pd.read_csv(StringIO(content_3))


# Academy: Business
response_4 = s3_client.get_object(Bucket='data-504-final-project-v2', Key='Academy_Combined/Business_combined.csv')
content_4 = response_4['Body'].read().decode('utf-8')
business = pd.read_csv(StringIO(content_4))

# Academy: Data
# Academy: Business
response_5 = s3_client.get_object(Bucket='data-504-final-project-v2', Key='Academy_Combined/Data_combined.csv')
content_5 = response_5['Body'].read().decode('utf-8')
data = pd.read_csv(StringIO(content_5))

# Academy: Engineering 
# Academy: Business
response_6 = s3_client.get_object(Bucket='data-504-final-project-v2', Key='Academy_Combined/Engineering_combined.csv')
content_6 = response_6['Body'].read().decode('utf-8')
engineering = pd.read_csv(StringIO(content_6))



In [121]:
applicants.head()

,id,name,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_date,invited_by
0,1,Esme Trusslove,Female,1994-08-04,etrusslove0@google.es,Swindon,22056 Lerdahl Avenue,SN1,4.429578e+11,"Saint George's Hospital Medical School, Univer...",2:1,2019-04-10,Bruno Bellbrook
1,2,Matthaeus Audas,Male,NaN,maudas1@mapquest.com,Charlton,263 Nelson Trail,OX12,4.495773e+11,Keele University,2:1,2019-04-30,Doris Bellasis
2,3,Cherey Tollfree,Female,1992-12-08,ctollfree2@netvibes.com,Weston,69 Coleman Court,GU32,4.458875e+11,"King's College London, University of London",2:1,2019-04-25,Gismo Tilling
3,4,Eryn Speers,Female,NaN,espeers3@shinystat.com,Sutton,0166 Daystar Drive,CT15,4.414879e+11,University of Edinburgh,2:1,NaN,NaN
4,5,Theadora Berkelay,Female,1995-11-03,tberkelay4@godaddy.com,Upton,6 Mandrake Crossing,WF9,4.484147e+11,University of Leicester,2:1,2019-04-02,Stacey Broad


### Dataframes - size

In [122]:
len(applicants)

4691

In [123]:
len(sparta_day)

4134

In [124]:
len(decision_scores)

989

In [125]:
len(business)+len(data)+len(engineering)

397

In [126]:
# Combine all academy names
academy = pd.concat([business[['name']], data[['name']], engineering[['name']]], ignore_index=True)

# Check 
len(academy) == len(business)+len(data)+len(engineering)

True

In [127]:
# Clean name fields for matching
applicants['name_clean'] = applicants['name'].str.strip().str.lower()
academy['name_clean'] = academy['name'].str.strip().str.lower()

In [128]:
# Merge academy students with applicant info
merged = academy.merge(applicants, on='name_clean', how='left')

In [129]:
merged.head()

,name_x,name_clean,id,name_y,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_date,invited_by
0,Quintus Penella,quintus penella,1769,Quintus Penella,Male,1992-11-11,qpenella7r@tripod.com,Whitchurch,8132 Grover Hill,BS14,4.414412e+11,NaN,NaN,2019-01-15,Bruno Bellbrook
1,Simon Murrey,simon murrey,1696,Simon Murrey,Male,1999-04-01,smurrey5q@pen.io,London,516 American Ash Drive,WC2H,NaN,University of Portsmouth,2:1,2019-01-16,Rupert Ripple
2,Gustaf Lude,gustaf lude,1663,Gustaf Lude,Male,1991-04-25,glude4t@harvard.edu,Newtown,392 Florence Pass,RG20,NaN,University of Bradford,3,2019-01-16,Rupert Ripple
3,Yolanda Fosse,yolanda fosse,1573,Yolanda Fosse,Female,1999-11-15,yfosse2b@over-blog.com,London,9683 Jenifer Street,EC1V,4.453733e+11,University of Wales,1,2019-01-16,Bruno Belbrook
4,Lynnett Swin,lynnett swin,1661,Lynnett Swin,Female,1996-12-29,lswin4r@icq.com,Carlton,0 Bellgrove Court,DL8,4.412476e+11,University of Greenwich,2:2,2019-01-16,Gismo Tilling


In [130]:
merged.isnull().sum()

name_x           0
name_clean       0
id               0
name_y           0
gender           8
dob             27
email           17
city             4
address          4
postcode         4
phone_number    33
uni             13
degree          13
invited_date    39
invited_by       0
dtype: int64

In [131]:
merged.dtypes

name_x           object
name_clean       object
id                int64
name_y           object
gender           object
dob              object
email            object
city             object
address          object
postcode         object
phone_number    float64
uni              object
degree           object
invited_date     object
invited_by       object
dtype: object

In [132]:
# Suppose 'id' is the unique identifier
merged_ids = merged['id'].unique()

### Critical info

In [133]:
required_cols = ['dob', 'email', 'city', 'address', 'postcode', 'phone_number']

In [134]:
# Critical info
# Only fill the critical info for applicants present in merged
for col in required_cols:
    mask = applicants['id'].isin(merged_ids)
    if applicants[col].dtype == 'object':
        applicants.loc[mask, col] = applicants.loc[mask, col].fillna('Required')
        merged[col] = merged[col].fillna('Required')
    else:
        applicants.loc[mask, col] = applicants.loc[mask, col].fillna(-1)
        merged[col] = merged[col].fillna(-1)



### Not essential: `invited_date`

In [135]:
# Invited date: label as 'not essential' 
mask = applicants['id'].isin(merged_ids)
applicants.loc[mask, 'invited_date'] = applicants.loc[mask, 'invited_date'].fillna('Not essential')
merged['invited_date'] = merged['invited_date'].fillna('Not essential')

### `gender`, `university`, `degree`

In [136]:
# Investigate university & gender
print("Gender categories:")
print(applicants['gender'].unique())

#print("\nUniversity categories:")
#print(applicants['uni'].unique())

print("\nDegree categories:")
print(applicants['degree'].unique())

Gender categories:
['Female' 'Male' nan]

Degree categories:
['2:1' '1' '3' nan '2:2']


In [137]:
# For gender: missing values -> 'Other/Missing'
# For uni & degree: most likely didn't attend -> 'Didn't attend/Missing'
for col, fill_value in zip(['gender', 'uni', 'degree'],
                           ['Other/Missing', "Didn't attend/Missing", "Didn't attend/Missing"]):
    applicants.loc[mask, col] = applicants.loc[mask, col].fillna(fill_value)
    merged[col] = merged[col].fillna(fill_value)


# Could just do 'clarify' for both


In [138]:
# Check 
print("Gender categories:")
print(merged['gender'].unique())

#print("\nUniversity categories:")
#print(merged['uni'].unique())

print("\nDegree categories:")
print(merged['degree'].unique())


Gender categories:
['Male' 'Female' 'Other/Missing']

Degree categories:
["Didn't attend/Missing" '2:1' '3' '1' '2:2']


### Checks

In [139]:
merged.isnull().sum()

name_x          0
name_clean      0
id              0
name_y          0
gender          0
dob             0
email           0
city            0
address         0
postcode        0
phone_number    0
uni             0
degree          0
invited_date    0
invited_by      0
dtype: int64

In [140]:
merged.dtypes

name_x           object
name_clean       object
id                int64
name_y           object
gender           object
dob              object
email            object
city             object
address          object
postcode         object
phone_number    float64
uni              object
degree           object
invited_date     object
invited_by       object
dtype: object

### Relabel the rest of applicant missing info as 'Not required'

In [141]:
applicants.isnull().sum()

id                0
name              0
gender          125
dob             206
email           254
city             90
address          90
postcode         90
phone_number    349
uni             150
degree          150
invited_date    897
invited_by      557
name_clean        0
dtype: int64

In [142]:
for col in applicants.columns:
    if applicants[col].dtype == 'object':
        applicants[col] = applicants[col].fillna('Not required')
    else:
        # keep phone number as NaN
        pass

In [143]:
applicants.isnull().sum()

id                0
name              0
gender            0
dob               0
email             0
city              0
address           0
postcode          0
phone_number    349
uni               0
degree            0
invited_date      0
invited_by        0
name_clean        0
dtype: int64

In [144]:
applicants.head()

,id,name,gender,dob,email,city,address,postcode,phone_number,uni,degree,invited_date,invited_by,name_clean
0,1,Esme Trusslove,Female,1994-08-04,etrusslove0@google.es,Swindon,22056 Lerdahl Avenue,SN1,4.429578e+11,"Saint George's Hospital Medical School, Univer...",2:1,2019-04-10,Bruno Bellbrook,esme trusslove
1,2,Matthaeus Audas,Male,Required,maudas1@mapquest.com,Charlton,263 Nelson Trail,OX12,4.495773e+11,Keele University,2:1,2019-04-30,Doris Bellasis,matthaeus audas
2,3,Cherey Tollfree,Female,1992-12-08,ctollfree2@netvibes.com,Weston,69 Coleman Court,GU32,4.458875e+11,"King's College London, University of London",2:1,2019-04-25,Gismo Tilling,cherey tollfree
3,4,Eryn Speers,Female,Not required,espeers3@shinystat.com,Sutton,0166 Daystar Drive,CT15,4.414879e+11,University of Edinburgh,2:1,Not required,Not required,eryn speers
4,5,Theadora Berkelay,Female,1995-11-03,tberkelay4@godaddy.com,Upton,6 Mandrake Crossing,WF9,4.484147e+11,University of Leicester,2:1,2019-04-02,Stacey Broad,theadora berkelay
